Imports:

In [1]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install scikit-learn
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn import linear_model as lm


<div style="text-align:center" >
<span style=font-size:60px>Data Retrieval</span>
</div>

In [2]:
dialogue = pd.read_csv(("https://raw.githubusercontent.com/"
 "rfordatascience/tidytuesday/master/data/"
 "2020/2020-09-08/friends.csv"))

print("Example Dialogue:")
dialogue.sample(4)


Example Dialogue:


,text,speaker,season,episode,scene,utterance
43058,"Oh, I forgot this was in here. Umm, this was t...",Rachel Green,7,6,12,14
28504,I'll take it all.,Gunther,5,6,2,22
47618,"Well, when Jake did it I saw that...he was wea...",Joey Tribbiani,7,22,7,6
38393,"But, I put some honey in it.",Monica Geller,6,16,2,8


In [10]:
episodes = pd.read_csv(("https://raw.githubusercontent.com/"
 "rfordatascience/tidytuesday/master/data/"
 "2020/2020-09-08/friends_info.csv"))

print("Example Episode:")
episodes.sample(4)

Example Episode:


,season,episode,title,directed_by,written_by,air_date,us_views_millions,imdb_rating
164,7,19,The One with Ross and Monica's Cousin,Gary Halvorson,Andrew Reich & Ted Cohen,2001-04-19,16.55,8.7
182,8,13,The One Where Chandler Takes a Bath,Ben Weiss,Vanessa McCarthy,2002-01-17,29.24,8.6
